# Heatmap matrix correlation between price and variables in Belgium
We are analyzing property data in Belgium and generating correlation heatmaps for different property types.
## Step 1: Import Required Libraries
We begin by importing the necessary libraries:

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np


## Step 2: Load the Data
Next, we load the property data from a CSV file:

In [2]:
df = pd.read_csv("../data/property_data.csv")

## Step 3: Data Cleaning
3.1. Removing Duplicates

To ensure each data point is unique, we remove duplicate entries:

In [3]:
df = df.drop_duplicates()

3.2. Removing Leading and Trailing Spaces
To maintain data consistency, we strip leading and trailing spaces:

In [4]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


3.3. Filling Missing Values
We replace missing values in numeric columns with '0' and in non-numeric columns with 'unknown':

In [5]:
df.loc[:, df.dtypes == np.float64] = df.loc[:, df.dtypes == np.float64].fillna(0)
df.loc[:, df.dtypes == np.int64] = df.loc[:, df.dtypes == np.int64].fillna(0)
df.loc[:, df.dtypes == object] = df.loc[:, df.dtypes == object].fillna('unknown')

## Step 4: Data Processing
4.1. Ensure Correct Data Types
We convert specific columns into appropriate data types:

In [6]:
df['Zip code'] = df['Zip code'].astype(int)
df['Subtype of property'] = df['Subtype of property'].astype(str)
df['Type of Sale'] = df['Type of Sale'].astype(str)
df['State of the building'] = df['State of the building'].astype(str)


4.2. One-Hot Encoding

We perform one-hot encoding on categorical variables, resulting in binary columns for each category:

In [7]:
df_encoded = pd.get_dummies(df, columns=['Subtype of property', 'Type of Sale', 'State of the building'])

## Step 6: Data Analysis
We group the data by 'Type of property' and conduct an analysis for each group. We skip the groups where type_of_property is '0' or 'unknown'. Depending on the type of property ('apartment' or other), we select a different set of numeric columns. We then calculate a correlation matrix for these numeric columns and visualize this matrix using a heatmap:

In [8]:
for type_of_property, group_df in df_encoded.groupby('Type of property'):
    if type_of_property == '0' or type_of_property.lower() == 'unknown':
        continue
    print(f"Type of property: {type_of_property}")

    if type_of_property == 'apartment':
        numeric_cols = ['Price of property in euro', 'Kitchen', 'Number of bedrooms', 'Living area', 'Terrace area', 'Garden', 'Garden area', 'Number of facades']
    else:
        numeric_cols = ['Price of property in euro', 'Kitchen', 'Number of bedrooms', 'Living area', 'Terrace area', 'Garden', 'Garden area', 'Surface of the land(or plot of land)', 'Number of facades', 'Swimming pool']

    for col in group_df.columns:
        if 'Subtype of property_' in col or 'Type of Sale_' in col or 'State of the building_' in col:
            numeric_cols.append(col)

    corr = group_df[numeric_cols].corr()
    fig = px.imshow(corr, title=f"Correlation matrix between price and variables for {type_of_property} in Belgium", zmin=-1, zmax=1)
    fig.show()


Type of property: apartment


Type of property: house


Price has the biggest correlation with variable living area and number of bedrooms because they are indicators of the size and capacity of the property. Larger and more spacious properties tend to have higher prices than smaller and more cramped ones. This means that larger and more spacious properties tend to have higher prices than smaller and more cramped ones.

Price has the smallest correlation with type of property appartement and type of sale under an option because they are not very relevant or distinctive factors for determining the price. Appartements are a common and diverse type of property that can have a wide range of prices depending on other features. Type of sale under an option is a temporary and conditional agreement that does not reflect the final or actual price of the property.
We find that price has the smallest correlation with type of property appartement and type of sale under an option, which are not very relevant or distinctive factors for determining the price. This means that these types do not have much influence on the price and can be ignored or excluded from our analysis.
	
Properties that need renovation tend to have lower prices than properties that are in good or new condition, because they require more work and investment from the buyer. Ground floor properties may have less privacy, security, or view than properties on higher floors, which may reduce their appeal and value.